In [32]:
import pandas
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import LinearIVGMM
from statsmodels.sandbox.regression.gmm import IV2SLS

df = pandas.read_stata('../Card1995.dta').dropna()

In [33]:
#lwage76, ed76 , age76, smsa76r, reg76r, nearc2, nearc4, nearc4a, nearc4b

#Set up X
y = df['lwage76']*1.0
X = pandas.DataFrame()
X['south'] = df['reg76r']*1.0
X['black'] = df['smsa76r']*1.0
X['exp'] = (1.0*df['age76'] - 6.0).add(-1.0*df['ed76'])
X['exp_sq'] = (X['exp']**2)/100

#Set up IV
I_a = pandas.DataFrame()
I_a = X.copy()
I_a['nearc4a'] = df['nearc4a']*1.0
I_a['nearc4b'] = df['nearc4b']*1.0

#Set Up Regular X
X_a = X.copy()
X_a['educ'] = df['ed76']
#X_a = sm.add_constant(X_a)

In [72]:
#part a - run standard model
model_a = IV2SLS(y,X_a,instrument=I_a).fit()
print '________2SLS Parameters________'
print model_a.params

________2SLS Parameters________
south     0.007390
black     0.217554
exp       0.382367
exp_sq   -1.326691
educ      0.291469
dtype: float64


In [71]:
#part b - add nearc2 as instrument
I_b = I_a.copy()
I_b['nearc2'] = df['nearc2']

model_b = IV2SLS(y,X_a,instrument=I_b).fit()
print model_b.params

south    -0.007523
black     0.163510
exp       0.335635
exp_sq   -1.075060
educ      0.308434
dtype: float64


In [70]:
#part c - even more insturments, reduced form
I_c = I_b.copy()
I_c['nearc4a*age76'] = I_b['nearc4a'].multiply(df['age76']*1.0)
I_c['nearc4a*age76^2/100'] = I_b['nearc4a'].multiply(df['age76']**2/100)

model_c = sm.OLS(X_a['educ'],I_c).fit()
print '________OLS Parameters________'
print model_b.params

________OLS Parameters________
south    -0.007523
black     0.163510
exp       0.335635
exp_sq   -1.075060
educ      0.308434
dtype: float64


In [69]:
#part d - 2SLS
I_c = I_b.copy()
I_c['nearc4a*age76'] = I_b['nearc4a'].multiply(df['age76']*1.0)
I_c['nearc4a*age76^2/100'] = I_b['nearc4a'].multiply(df['age76']**2/100)
model_c = IV2SLS(y,X_a,I_c).fit()
print '________2SLS Parameters________'
print model_c.params

________2SLS Parameters________
south     0.005668
black     0.211316
exp       0.376973
exp_sq   -1.297649
educ      0.293427
dtype: float64


In [1]:
#part e - LIML package doesn't exist yet...

/home/erichschulman/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [68]:
#12.21 part a
model_1221_a = LinearIVGMM(y,X_a,I_a)
model_1221_a = LinearIVGMM(y,X_a,I_a).fitgmm(model_1221_a.fitstart())
print '________Linear GMM Parameters________'
print model_1221_a

________Linear GMM Parameters________
[ 0.0073895   0.2175537   0.38236655 -1.32669071  0.29146924]


________Linear GMM Parameters________
[-0.00752271  0.16350964  0.33563465 -1.07505988  0.30843416]
